<a href="https://colab.research.google.com/github/sphuang28/Tibaml11103-Teamproject/blob/main/%E9%A0%90%E6%B8%AC%E7%A8%8B%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install googlemaps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import geopy.distance
from google.colab import files
from google.colab import drive
import googlemaps

# 使用者輸入資訊

In [ ]:
def user_input():
  
  pre_f = pd.DataFrame(columns=["土地位置建物門牌", "建物型態", "有無管理組織", "移轉層次", 
                  "車位類別", "鄉鎮市區", "電梯", "屋齡", "建物移轉總坪數"], index=[0])

  dict = {"有":1, "無":0}

  b_dict = {"住宅大樓":"住宅大樓(11層含以上有電梯)", "公寓": "公寓(5樓含以下無電梯)", "套房": "套房(1房1廳1衛)", 
          "華廈": "華廈(10層含以下有電梯)", "透天厝": "透天厝"}

  # address = input()
  # b_type = b_dict[input()]
  # manage = dict[input()]
  # floor = int(input())
  # p_type = input()
  # dist = input()
  # ele = dict[input()]
  # age = int(input())
  # trans = float(input())

  pre_f["土地位置建物門牌"][0] = "臺北市中山區民生東路三段36號"  # address
  pre_f["建物型態"][0] = "華廈"  # b_type
  pre_f["有無管理組織"][0] = 1  # manage
  pre_f["移轉層次"][0] = 3  # floor
  pre_f["車位類別"][0] = "無車位"  # p_type
  pre_f["鄉鎮市區"][0] = "中山區"  # dist
  pre_f["電梯"][0] = 1  # ele
  pre_f["屋齡"][0] = 30  # age
  pre_f["建物移轉總坪數"][0] = 20 # trans

  return pre_f

# 新增經緯度

In [1]:
def lat_lng():

  pre_f["緯度"] = 0.0
  pre_f["經度"] = 0.0

  for i in range(len(pre_f)):  
    gmaps = googlemaps.Client(key="") # 輸入google maps API 金鑰
    address = pre_f["土地位置建物門牌"][i]
    geocode = gmaps.geocode(address=address)
    (lat, lng) = map(geocode[0]['geometry']['location'].get, ('lat', 'lng'))

    pre_f["緯度"][i] = (lat, lng)[0]
    pre_f["經度"][i] = (lat, lng)[1]

# 新增設施

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Tibame_03期_第二組團專/整理後的資料/每年每季房屋交易資料') #切換該目錄
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['設施座標表Ver2.xlsx',
 '台北市房屋交易_111S1.csv',
 '台北市房屋交易_all_1125_2.csv',
 '台北市房屋交易_all_1125_3.csv',
 '台北市房屋交易_111_ALL.csv',
 '台北市房屋交易_all_standard&Normal_1125.csv',
 '台北市房屋交易_111_standard&Normal_model.csv',
 '台北市房屋交易_all_standard&Normal_1125_2.csv',
 '台北市房屋交易_all_正規化_1126.csv',
 '台北市房屋交易_for_model_正規化_1126.csv',
 '台北市房屋交易_for_model_1126_加設施.csv',
 '台北市房屋交易_all_正規化_1126_2.csv',
 '台北市房屋交易_all_未正規化_1126_3.csv',
 '台北市房屋交易_all_未正規化_1126_4.csv',
 '台北市房屋交易_for_model_1126_加設施_2.csv',
 '台北市房屋交易_all_未正規化_1126_5.csv',
 '台北市房屋交易_for_model_1126_加設施_3.csv',
 '台北市房屋交易_測試用10筆資料.csv',
 'catboost_info',
 '台北市房屋交易_model_1130_未正規化.csv',
 '台北市房屋交易_model_1130_未正規化_2.csv',
 '台北市房屋交易_model_1130.csv',
 'reg_model.sav']

In [ ]:
def park():
  
  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="公園") 

  pre_f["公園數量"]=""
  pre_f["固定範圍(公園)"]=3000/1000
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 3000/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"公園數量"] = times

def mu():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="博物館")  

  pre_f["固定範圍(博物館)"]=10000/1000
  pre_f["博物館數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 10000/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"博物館數量"] = times

def lib():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="圖書館")  

  pre_f["固定範圍(圖書館)"]=5000/1000
  pre_f["圖書館數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 5000/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"圖書館數量"] = times

def mrt():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="捷運出口")  

  pre_f["固定範圍(捷運出口)"]=500/1000
  pre_f["捷運出口數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 500/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"捷運出口數量"] = times

def edu():
  
  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="學校") 

  pre_f["固定範圍(學校)"]=500/1000
  pre_f["學校數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 500/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"學校數量"] = times

def fire():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="消防") 

  pre_f["固定範圍(消防局)"]=2000/1000
  pre_f["消防局數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 2000/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"消防局數量"] = times

def dead():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="禮儀")  

  pre_f["固定範圍(禮儀)"]=5000/1000
  pre_f["禮儀數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 5000/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"禮儀數量"] = times

def pol():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="警察")  

  pre_f["固定範圍(警察局)"]=500/1000
  pre_f["警察局數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 500/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"警察局數量"] = times

def post():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="郵局") 

  pre_f["固定範圍(郵局)"]=500/1000
  pre_f["郵局數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 500/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"郵局數量"] = times

def fin():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="金融")  

  pre_f["固定範圍(金融)"]=500/1000
  pre_f["金融數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 500/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"金融數量"] = times

def hosp():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="醫療醫院")  

  pre_f["固定範圍(醫院)"]=1000/1000
  pre_f["醫院數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 1000/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"醫院數量"] = times

def clinic():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="醫療診所")  

  pre_f["固定範圍(診所)"]=500/1000
  pre_f["診所數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 500/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"診所數量"] = times

def pharmacy():

  dh = pd.read_excel("設施座標表Ver2.xlsx", sheet_name="醫療藥局")  

  pre_f["固定範圍(藥局)"]=500/1000
  pre_f["藥局數量"]=""
  for i in range(len(pre_f)):
      times = 0
      origin_lat = str(pre_f["緯度"][i])
      origin_long = str(pre_f["經度"][i])
      origin = (origin_lat, origin_long)
      for j in range(len(dh)):
          destination_lat = str(dh["lat"][j])
          destination_long = str(dh["lng"][j])
          destination = (destination_lat, destination_long)
          distance=geopy.distance.geodesic(origin, destination).km
          try:
              if distance <= 500/1000:
                  times = times + 1
                  
          except:
              print(i,"error")
      pre_f.at[i,"藥局數量"] = times

# 載入模型進行預測

In [ ]:
pd.set_option('display.max_columns', 100)
pre_f

,土地位置建物門牌,移轉層次,屋齡,建物移轉總坪數,緯度,經度,公園數量,固定範圍(公園),固定範圍(博物館),博物館數量,固定範圍(圖書館),圖書館數量,固定範圍(捷運出口),捷運出口數量,固定範圍(學校),學校數量,固定範圍(消防局),消防局數量,固定範圍(禮儀),禮儀數量,固定範圍(警察局),警察局數量,固定範圍(郵局),郵局數量,固定範圍(金融),金融數量,固定範圍(醫院),醫院數量,固定範圍(診所),診所數量,固定範圍(藥局),藥局數量,鄉鎮市區_中山區,建物型態_華廈,車位類別_無車位,有無管理組織_1,電梯_1
0,臺北市中山區民生東路三段36號,3,30,20,25.057592,121.54053,175,3.0,10.0,35,5.0,39,0.5,0,0.5,1,2.0,6,5.0,2,0.5,0,0.5,2,0.5,27,1.0,1,0.5,71,0.5,25,1,1,1,1,1


類別資料要做one-hot處理！！！！

In [ ]:
def pred():
  reg = pickle.load(open("reg_model.sav", "rb"))  # 載入模型

  pre_f = pre_f.drop(["土地位置建物門牌", "緯度", "經度"], axis=1)
  pre_f = pd.get_dummies(pre_f, columns = ["鄉鎮市區", "建物型態", "車位類別", "有無管理組織", "電梯"])

  y_pre = reg.predict(pre_f)  # 進行預測

  # 反正規化處理，求預測價格

  p_max = 84100000
  p_min = 5300000

  p_total_price = y_pre * (p_max - p_min) + p_min  # 預測的價格
  (p_total_price / pre_f["建物移轉總坪數"][0]) / 10000  # 單價： 萬/坪

# 主程式

In [ ]:
# 主程式

# 使用者輸入資訊
user_input()

#=========================

# 新增經緯度
lat_lng()

#=========================

# 新增距離設施
park()
mu() 
lib() 
mrt() 
edu() 
fire() 
dead() 
pol() 
post()
fin()  
hosp()  
clinic()
pharmacy()

#=========================

# 預測價錢
pred()

<ipython-input-80-afd672b41bf0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_f["緯度"][i] = (lat, lng)[0]
<ipython-input-80-afd672b41bf0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_f["經度"][i] = (lat, lng)[1]
